# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from pprint import pprint

# Import API key
from api_keys import g_key
gmaps.configure(api_key=g_key)

#output file path for data resource files and figures
outputFilePath = os.path.join("Output-Data")

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
#read the weather_cities.csv into a dataframe
weather_cities_df = pd.read_csv(os.path.join("Output-Data","weather_cities.csv"),delimiter=",")
weather_cities_df

,Latitude,Longitude,City,Country,Max Temp,Humidity,Wind Speed,Cloudiness,Date
0,65.99,57.53,usinsk,ru,40.41,82.0,7.83,42.0,1.601230e+09
1,77.48,-69.36,qaanaaq,gl,14.20,78.0,6.93,0.0,1.601230e+09
2,-21.36,55.77,saint-philippe,re,69.80,64.0,5.82,84.0,1.601230e+09
3,71.00,24.66,havoysund,no,44.60,69.0,11.41,0.0,1.601230e+09
4,-21.01,55.27,saint-paul,re,69.80,88.0,8.05,88.0,1.601230e+09
...,...,...,...,...,...,...,...,...,...
684,10.99,-74.95,puerto colombia,co,87.80,70.0,4.70,40.0,1.601230e+09
685,-1.20,-46.02,carutapera,br,85.86,61.0,10.67,35.0,1.601231e+09
686,-34.05,23.37,plettenberg bay,za,52.16,69.0,3.91,14.0,1.601231e+09
687,52.90,16.56,czarnkow,pl,52.00,93.0,3.36,100.0,1.601231e+09


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
#store locations by latitude and longitude
locations = weather_cities_df[["Latitude","Longitude"]].astype(float)

#drop na values (though these should have been handled in WeatherPy before saving to csv)
#do not rid of the index though - indexes will correspond between locations and humididty
weather_cities_df = weather_cities_df.dropna()

#convert humidity to float
humidity = weather_cities_df["Humidity"].astype(float)
maxHumidity = humidity.max()

In [6]:
#create a humidity heatmap layer
fig = gmaps.figure()

heatLayer_humidity = gmaps.heatmap_layer(locations, weights = humidity, dissipating=False, max_intensity = maxHumidity, point_radius = 3)

fig.add_layer(heatLayer_humidity)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [7]:
#set criteria for vacation weather
vacationWeather_df = weather_cities_df.loc[(weather_cities_df["Max Temp"]>70)&
                                           (weather_cities_df["Max Temp"]<80)&
                                           (weather_cities_df["Cloudiness"]==0)&
                                           (weather_cities_df["Wind Speed"]<10)].reset_index(drop=True)
vacationWeather_df

,Latitude,Longitude,City,Country,Max Temp,Humidity,Wind Speed,Cloudiness,Date
0,-20.32,118.57,port hedland,au,71.60,83.0,9.17,0.0,1.601230e+09
1,-17.78,15.68,oshakati,na,73.80,19.0,4.56,0.0,1.601230e+09
2,39.05,-27.98,lagoa,pt,71.49,86.0,5.26,0.0,1.601230e+09
3,24.83,-104.83,durango,mx,76.28,22.0,3.53,0.0,1.601230e+09
4,27.98,-114.06,guerrero negro,mx,77.83,56.0,9.13,0.0,1.601230e+09
5,38.40,33.41,eskil,tr,73.96,33.0,6.44,0.0,1.601230e+09
6,29.58,-9.50,tiznit,ma,73.02,51.0,5.28,0.0,1.601230e+09
7,21.65,-101.50,ocampo,mx,73.11,31.0,7.78,0.0,1.601231e+09


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [8]:
#create dataframe for the hotel information
hotel_df = vacationWeather_df.loc[:,["Latitude","Longitude","City","Country"]]
# hotel_df = pd.DataFrame(hotel_df)
hotel_df

,Latitude,Longitude,City,Country
0,-20.32,118.57,port hedland,au
1,-17.78,15.68,oshakati,na
2,39.05,-27.98,lagoa,pt
3,24.83,-104.83,durango,mx
4,27.98,-114.06,guerrero negro,mx
5,38.40,33.41,eskil,tr
6,29.58,-9.50,tiznit,ma
7,21.65,-101.50,ocampo,mx


In [9]:
#set up lists for hotel information
hotelName = []
hotelAddress = []
hotelRating = []
hotelLat = []
hotelLng = []

for index, row in hotel_df.iterrows():
    #current info
    currentCity = row["City"]
    currentCountry = row["Country"]
    
    #geocoordinates
    target_coordinates = f"{row['Latitude']},{row['Longitude']}"
    target_radius = 5000 #this is in meters
    target_type = "hotel"
    target_keyword = "hotel"

    #set parameters
    params = {
        "location": target_coordinates,
        "radius": target_radius,
        "type": target_type,
        "keyword": target_keyword,
        "key": g_key
    }
    
    #base url
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    #run request using params dictionary
    response = requests.get(base_url,params=params).json()
    #simplify finding results
    results = response['results']
     
    try:
        hotelName.append(results[0]['name'])
        hotelAddress.append(results[0]['vicinity'])
        hotelRating.append(results[0]['rating'])
        hotelLat.append(results[0]['geometry']['location']['lat'])
        hotelLng.append(results[0]['geometry']['location']['lng'])
        print(f"Processeing Record for a {target_keyword} in {currentCity}, {currentCountry}")
    except:
        hotelName.append(np.nan)
        hotelAddress.append(np.nan)
        hotelRating.append(np.nan)
        hotelLat.append(np.nan)
        hotelLng.append(np.nan)
        print(f"No results/{target_keyword} found in {currentCity}, {currentCountry}. Skipping...")    

Processeing Record for a hotel in port hedland, au
Processeing Record for a hotel in oshakati, na
Processeing Record for a hotel in lagoa, pt
No results/hotel found in durango, mx. Skipping...
Processeing Record for a hotel in guerrero negro, mx
No results/hotel found in eskil, tr. Skipping...
No results/hotel found in tiznit, ma. Skipping...
Processeing Record for a hotel in ocampo, mx


In [10]:
#add new information into the cities_df
hotel_df["Hotel Name"] = hotelName
hotel_df["Hotel Address"] = hotelAddress
hotel_df["Hotel Rating"] = hotelRating
hotel_df["Hotel Latitude"] = hotelLat
hotel_df["Hotel Longitude"] = hotelLng

hotel_df

,Latitude,Longitude,City,Country,Hotel Name,Hotel Address,Hotel Rating,Hotel Latitude,Hotel Longitude
0,-20.32,118.57,port hedland,au,The Esplanade Hotel,"2 Anderson St, Port Hedland",4.0,-20.313925,118.576081
1,-17.78,15.68,oshakati,na,Oshandira Lodge,Oshakati,3.9,-17.792517,15.698013
2,39.05,-27.98,lagoa,pt,Graciosa Resort - Biosphere Island Hotel,"Porto da Barra, Santa Cruz da Graciosa",4.4,39.085412,-27.997393
3,24.83,-104.83,durango,mx,NaN,NaN,NaN,NaN,NaN
4,27.98,-114.06,guerrero negro,mx,Hotel Los Caracoles,"Calzada, De La República s/n, Estado 30, Guerr...",4.1,27.967758,-114.028336
5,38.40,33.41,eskil,tr,NaN,NaN,NaN,NaN,NaN
6,29.58,-9.50,tiznit,ma,NaN,NaN,NaN,NaN,NaN
7,21.65,-101.50,ocampo,mx,HOTEL OCAMPO,"CARRETERA OCAMPO-LEON KM 1, Ocampo",0.0,21.635853,-101.472675


In [11]:
#remove NaN rows
#did not drop indexes for locations without hotel found - can see how many hotels are found for top locations
hotelFound_df = hotel_df.dropna()
hotelFound_df

,Latitude,Longitude,City,Country,Hotel Name,Hotel Address,Hotel Rating,Hotel Latitude,Hotel Longitude
0,-20.32,118.57,port hedland,au,The Esplanade Hotel,"2 Anderson St, Port Hedland",4.0,-20.313925,118.576081
1,-17.78,15.68,oshakati,na,Oshandira Lodge,Oshakati,3.9,-17.792517,15.698013
2,39.05,-27.98,lagoa,pt,Graciosa Resort - Biosphere Island Hotel,"Porto da Barra, Santa Cruz da Graciosa",4.4,39.085412,-27.997393
4,27.98,-114.06,guerrero negro,mx,Hotel Los Caracoles,"Calzada, De La República s/n, Estado 30, Guerr...",4.1,27.967758,-114.028336
7,21.65,-101.50,ocampo,mx,HOTEL OCAMPO,"CARRETERA OCAMPO-LEON KM 1, Ocampo",0.0,21.635853,-101.472675


In [12]:
print(f"There were {len(hotel_df)} locations matching vacation requirements and {len(hotelFound_df)} hotels found.")

There were 8 locations matching vacation requirements and 5 hotels found.


In [13]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotelFound_info = [info_box_template.format(**row) for index, row in hotelFound_df.iterrows()]
locations = hotelFound_df[["Hotel Latitude", "Hotel Longitude"]]

In [15]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content=hotelFound_info)
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))